In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data Preprocessing**
Let's import libraries like pandas,numpy and cv2 for preprocess datas

In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.layers import MaxPooling2D,AveragePooling2D,Conv2D,Flatten,Dropout,BatchNormalization
from keras.layers.core import Dense
from keras.utils import to_categorical,np_utils
import keras
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd

Read train and test csv

In [ ]:
df=pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test=pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

**Now let's reshape datas and put it in x and y training variables.**
* We will use them func to_categorical() for create dummy variables
>  from y=
>  [1,
>   2,
>   1]
>  to y1=
>  [
>   [1,0],
>   [0,1],
>   [1,0]
>  ]

In [ ]:
y=df['label']
y_test=test['label']
del df['label']
del test['label']
x=df.to_numpy()
x=x.reshape(-1,28,28).astype('float32')
y_train=np_utils.to_categorical(y,25)
y_test=keras.utils.to_categorical(y_test,25)


*Let's see shapes*

In [ ]:
x.shape

In [ ]:
y.shape

# **Transform 28x28 to 32x32 with OpenCv**

*When i studied i try first to use Le-net5 and it want input by 32x32, them i modify the architecture but i use same input shape*

In [ ]:
x_train=[]
for x1 in x:
    x_train.append(cv2.resize(x1,(32,32)))
x_test=test.to_numpy().reshape(-1,28,28).astype('float32')
x_test_resh=[]
for x1 in x_test:
    x_test_resh.append(cv2.resize(x1,(32,32)))
x_test=x_test_resh

Normally arrays are in range 0-255 but the  architecture perform well when range is 0-1

In [ ]:
x_train=np.asarray(x_train).reshape(-1,32,32,1)
x_train/=255
x_test=np.asarray(x_test).reshape(-1,32,32,1)
x_test/=255

# **LeCunn LeNet-5 Architecture**

This model have 113k params

In [ ]:

model= Sequential()
model.add(Conv2D(6,(5,5),1,input_shape=(32,32,1)))
model.add(AveragePooling2D(2,2))
model.add(Conv2D(32,(5,5),1,padding='valid'))
model.add(AveragePooling2D(2,2))
model.add(Flatten())
model.add(Dense(120,activation='relu'))
model.add(Dense(84,activation='relu'))
model.add(Dense(25,activation='softmax'))
model.summary()


# **My architecture**

This model have 78k params

In [ ]:
del model
model= Sequential()
model.add(Conv2D(4,(5,5),1,input_shape=(32,32,1),padding='same'))
model.add(BatchNormalization())

model.add(MaxPooling2D(2,2))

model.add(Dropout(0.20))

model.add(Conv2D(16,(5,5),1,padding='valid'))
model.add(BatchNormalization())

model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(128,activation='relu'))

model.add(Dense(25,activation='softmax'))
model.summary()

I used a scheduler and setting up a optimizer like Adam and them more test i see 25 epochs let me 92% of accuracy in test data

In [ ]:
def scheduler(epoch, lr):
  return round(0.003 * 1/(1 + 0.319 * epoch), 10) # change 2
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.003),
    metrics=['accuracy']
)
history=model.fit(x_train,y_train,verbose=1,epochs=20,batch_size=256,callbacks=[LearningRateScheduler(scheduler, verbose=1)])

*Let's see data test shape*

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Let's show a graph.

In [ ]:
plt.plot(history.history['loss'],label='Train Loss')
plt.plot(history.history['accuracy'],label='Train Accuracy')
plt.legend()
plt.show()

*Thanks for read my notebook, write me a feedback for future works <3*